## Example of extracting kmers from fasta files

Not sure format the fasta files will come in.   
But have read sequences into a dictionary where the key will be some identifier eg. accesion number 

I assume we will use  some sort of CVS file with sequence ID and ARM info (label).
Then assume that there is equivelent fasta,faa file of CDS and pfs file with domains with same identifier in appropriate folders. ../data/fasta , ../data/faa and ../data/pfs
#### Need to do a check for missing sequences before start. 

In [4]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import pickle
import random
import numpy as np
from collections import defaultdict, Counter
from Bio import SeqIO

import sys
lib_dir = '../mylibs'
if lib_dir not in sys.path:
    sys.path.append(lib_dir)
    



In [92]:
def extract_kmers (sequences,k,symbol_dict):
    
    vocab_len =  symbol_dict['mod'] **k
    f_index = fe.get_feature_names(symbol_dict,k)
    seq_index = [None]
    X = np.zeros((len(sequences) ,vocab_len))
    
    for i,(accn, seq) in enumerate (sequences.items()):
        X[i] = get_word_frequ (seq,k, vocab_len,symbol_dict)
        
# normalise for length of genome(s)
    XN = np.divide(X,X.sum(axis = 1)[:,None])
    print (len (seq_index),np.shape(XN))
    
    return XN,seq_index,f_index
    

def get_word_frequ(sequence,k,vocab_len,symbol_dict):
    """returns the word frequency array for all genomes for a virus """
    word_freqs = np.zeros(shape =(vocab_len),dtype=np.int32)
    for i in range(len(sequence)-k+1):
        word = sequence[i:i+k]
        if check_word(word,symbol_dict):
            kmer_index = patternToNumber (word,k, symbol_dict)
            word_freqs [ kmer_index] += 1
            # If this for DNA/RNA then add the k-mers complinent
#             if symbol_dict['mod'] == 4:
#                 word_freqs [patternToNumber(compliment(word),k,symbol_dict)] +=1
    print(word_freqs)
                 
    return word_freqs



def symbolToNumber (symbol,symbols):
    return symbols[symbol]


def patternToNumber (pattern,k,symbol_dict):
    mod = symbol_dict['mod']
    number = 0
    for i in range (0,k):
        n = symbolToNumber ( pattern [k-1-i],symbol_dict)
        number += n* mod**i 
    return number

def get_feature_names(symbols,k):
    all_kmers = []
    n2sym = { v:k for k,v in symbols.items()}
    mod = symbols['mod']
    for number in range(mod**k):
        pattern = ''
        for i in range (0,k):
            n = number% mod
            number = (number - n)/mod 
            symbol = n2sym[n]
            pattern = symbol + pattern
            print (pattern,number,symbol,n)

        all_kmers.append (pattern)
    return all_kmers



def check_word(mystring,symbols):
    """Check for illegal characters """
    return all(c in symbols for c in mystring)

def compliment(dna):
    """Return the reverse compliment of a sequence """
    bp = { 'a':'t', 't':'a', 'c':'g','g':'c'}
    revdna =''    
    for i in range(len(dna)-1,-1,-1):
         revdna += bp[dna[i]]
    return (revdna)

### Setup functions for extracting features,list of kmer lengths, file paths for fasta/pfs(domain), and the symbol dictionaries


In [94]:

# Input file with list of reseq ids and labels

genome_list = 'bact.csv'

# Everthing in the same order as the feature_function list
# a list of the functions to be used create the feature matrix

features = ['DNA']#,'AA', 'PC', 'Domains']

# filepath for the fasta, faa and domain files
filepaths = [
                '../data/fasta']
#                 ,'/home4/youn01f/Desktop/workspace/newData/faa','/home4/youn01f/Desktop/workspace/newData/faa',
#                 '/home4/youn01f/Desktop/workspace/newData/pfs',]

# A list of kmers lengths for each feature set to be tested
kmer_lists = [
              [1]]
#,2,3,4,]]
#     5,6,7,8,9], # dna 
#               [1,2,3,4], # aa
#               [1,2,3,4,5,6] ,    #pc
#               [1]] #domains

# symbol dictionaries 
na_dict = {'mod':4,'a':0,'c':1,'g':2,'t':3}
aa_dict = {'mod':20 ,'A':0,'C':1,'D':2,'E':3,'F':4,'G':5,'H':6,'I':7,'K':8,'L':9,'M':10,'N':11,
              'P':12,'Q':13,'R':14,'S':15,'T':16,'V':17,'W':18,'Y':19}
pc_dict = {'mod':7, 'C':0,
            'A':1,'G':1,'V':1,
           'I':2,'L':2,'F':2,'P':2,
           'M':3,'S':3,'T':3,'Y':3,
           'H':4,'N':4,'Q':4,'W':4,
           'R':5,'K':5,
           'D':6,'E':6}
symbol_dicts = [
                na_dict] 
#     ,aa_dict,pc_dict, 
#                 {'mod':1}]



### Mini test

In [95]:
seqs = {'a': 'actggttaac', 'b': 'ggggggtttttt'}
X, seq_index, f_index = extract_kmers (seqs,1,symbol_dict)
print (X)

[3 2 2 3]
[0 0 6 6]
1 (2, 4)
[[0.3 0.2 0.2 0.3]
 [0.  0.  0.5 0.5]]


### Read fasta file downloaded from Patric

In [96]:
fastafile = '../data/fasta/PATRIC_genome_sequence.fasta'
count = 0
sequences = {}
with open(fastafile, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
            sequences[record.id]= record.seq
            print (record.id, len(record.seq))
                   
print (len(sequences))

accn|AUYU01000009 453616
accn|AUZB01000012 275568
accn|CP019569 14755
accn|CP016281 62833
accn|CP016090 6123550
accn|CP018624 2872664
6


### Main Loops 
Loop over the different features and different k's.  
I did this like this because I was running this on multipe datasets.  
We Could probally simplify this.  

In [100]:
# 


for index,feature in enumerate(features):
    kmers = kmer_lists[index]
    symbol_dict = symbol_dicts[index]
    for k in kmers:
        if feature != 'Domains':
            print (f'Extacting  {feature_func.__name__}  of length  {k}  from   {len(sequences)} sequences')
            X, seq_index, f_index = extract_kmers (sequences,k,symbol_dict)

            


Extacting  DNA_features  of length  1  from   6 sequences
[141782  80802  48771 182261]
[114279  28867  41819  90603]
[5012 1980 1885 5878]
[20996  9973  7370 24494]
[2159590  909743  919261 2134956]
[1010166  420812  391597 1050089]
1 (6, 4)
